## Homework: Vector Search
In this homework, we will learn more about vector search and embedding. Like in the module, we will use Qdrant and fastembed

### Embeddings
Qdrant uses fastembed under the hood to turn text into vectors.

## Q1. Embedding the query
Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51

In [ ]:
!pip install fastembed qdrant-client numpy pandas

In [ ]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient("http://localhost:6333")
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"




In [20]:
import numpy as np
query = 'I just discovered the course. Can I join now?'
# vector_query = models.Document(text=query, model=model_handle)
# print(np.shape(vector_query))


import numpy as np
from fastembed import TextEmbedding
embedder = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')


q, = list(embedder.embed(query))
np.shape(q)

print("Minimum value: ", q.min())


Minimum value:  -0.11726373885183883


#### So the minimum value  = -0.11

#### Cosine similarity
The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the norm function


```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

In [21]:
import numpy as np
np.linalg.norm(q)




np.float64(1.0)

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query and the vector for the document?

* 0.3

* 0.5

* 0.7

* 0.9

In [23]:
q_emb, = list(embedder.embed(query))
doc = 'Can I still join the course after the start date?'
doc_embed, = list(embedder.embed(doc))

doc_embed.dot(q_emb)

np.float64(0.9008528895674548)

#### So the cosine similarity is 0.9

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python

documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

```



Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

* 0
* 1
* 2
* 3
* 4

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free to ignore it.

In [25]:
import pandas as pd

documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]


df = pd.DataFrame(documents)
df

,text,section,question,course
0,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,data-engineering-zoomcamp
1,"Yes, we will keep all the materials after the ...",General course-related questions,Course - Can I follow the course after it fini...,data-engineering-zoomcamp
2,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,data-engineering-zoomcamp
3,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,data-engineering-zoomcamp
4,Star the repo! Share it with friends if you fi...,General course-related questions,How can we contribute to the course?,data-engineering-zoomcamp


In [ ]:
# Q3: rank documents by cosine similarity to the query
import numpy as np
import pandas as pd

# Assumes you already have: `query`, `embedder`, and `df` (built from `documents`)
# Embed the query
q_vec, = list(embedder.embed([query]))

# Embed all questions
questions = df['question'].tolist()
Q = np.array(list(embedder.embed(questions)), dtype=np.float32)

# Cosine similarity = dot of L2-normalized vectors
Qn = Q / np.linalg.norm(Q, axis=1, keepdims=True)
qn = q_vec / np.linalg.norm(q_vec)

scores = Qn @ qn

df_q3 = df.copy()
df_q3['score'] = scores
df_q3 = df_q3.sort_values('score', ascending=False).reset_index(drop=True)

# Show top results
df_q3[['score', 'question', 'section']].head(10)


## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:

full_text = doc['question'] + ' ' + doc['text']
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

* 0
* 1
* 2
* 3
* 4

Is it different from Q3? If yes, why?

In [ ]:
# Q4: single-matrix approach + top-k
import numpy as np
import pandas as pd

# Reuse Qn and qn from above; rebuild if not in scope
if 'Qn' not in globals() or 'qn' not in globals():
    questions = df['question'].tolist()
    Q = np.array(list(embedder.embed(questions)), dtype=np.float32)
    Qn = Q / np.linalg.norm(Q, axis=1, keepdims=True)
    q_vec, = list(embedder.embed([query]))
    qn = q_vec / np.linalg.norm(q_vec)

scores = Qn @ qn

topk = 10
idx = np.argpartition(-scores, topk)[:topk]
idx = idx[np.argsort(-scores[idx])]

df_q4 = df.iloc[idx].copy()
df_q4['score'] = scores[idx]
df_q4 = df_q4.sort_values('score', ascending=False).reset_index(drop=True)

df_q4[['score', 'question', 'section']].head(topk)


## Q5. Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

* 128
* 256
* 384
* 512

One of these models is BAAI/bge-small-en. Let's use it.

In [ ]:
# Q5: choose the embedding model (fast vs accuracy)
# If you're already using fastembed earlier, this just (re)sets the model explicitly.

from fastembed import TextEmbedding

# Option A (fast, solid quality): 
model_handle = "jinaai/jina-embeddings-v2-small-en"

# Option B (better quality, a bit slower):
# model_handle = "jinaai/jina-embeddings-v2-base-en"

embedder = TextEmbedding(model_name=model_handle)
print("Using embedding model:", model_handle)


## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:


```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67



In [ ]:
# Q6: index the docs into an in-memory Qdrant, then query with the Q1 question
from qdrant_client import QdrantClient, models
import numpy as np
import pandas as pd
import uuid

# Prepare vectors for all documents (index the 'question', keep full payload)
questions = df['question'].tolist()
texts = df['text'].tolist()
sections = df['section'].tolist()
courses = df['course'].tolist()

vectors = np.array(list(embedder.embed(questions)), dtype=np.float32)
dim = vectors.shape[1]

# In-memory Qdrant instance
client = QdrantClient(location=":memory:")
collection = "hw2_docs"

client.recreate_collection(
    collection_name=collection,
    vectors_config=models.VectorParams(size=dim, distance=models.Distance.COSINE),
)

# Upsert points
points = [
    models.PointStruct(
        id=str(uuid.uuid4()),
        vector=vectors[i].tolist(),
        payload={
            "question": questions[i],
            "text": texts[i],
            "section": sections[i],
            "course": courses[i],
        },
    )
    for i in range(len(questions))
]
client.upsert(collection_name=collection, points=points)

# Query with the same `query` from Q1
q_vec, = list(embedder.embed([query]))

results = client.search(
    collection_name=collection,
    query_vector=q_vec.tolist(),
    limit=10,
    with_payload=True
)

# Report highest score (first returned record) and show a small table
if results:
    top = results[0]
    print("Top score:", round(top.score, 4))
    print("Top question:", top.payload.get("question"))

    out = pd.DataFrame([{
        "score": r.score,
        "question": r.payload.get("question"),
        "section": r.payload.get("section")
    } for r in results]).sort_values("score", ascending=False).reset_index(drop=True)

    out.head(10)
else:
    print("No results returned.")
